In [1]:
# --- Colab setup: run this cell once ---
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install geopandas rasterio shapely fiona pyproj

# Make labels
The labels are a mask of the AOI, with one value per pixel. A 1 means that pixel was occupied by a wolf pack, and everywhere else is 0.

In [ ]:
from pathlib import Path

import geopandas as gpd
import rasterio
from rasterio.features import rasterize

DRIVE_ROOT = Path("/content/drive/MyDrive")

COMPOSITE_DIR = DRIVE_ROOT / "WolfCast_Satellite_Data"

SHAPE_ROOT = DRIVE_ROOT / "1995_2022-YELL-wolf-data"

OUTPUT_DIR = DRIVE_ROOT / "WolfCast_Satellite_Data_labels"

START_YEAR = 1995
END_YEAR = 2022

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def extract_year_from_path(path: Path):
    """
    Best-effort year detection based on the known directory/filename patterns
    in 1995_2022-YELL-wolf-data.
    """
    s = str(path)
    filename = path.stem

    import re
    filename_patterns = [
        r"^(\d{4})_\w+",      # 2008_agate_mcp
        r"ter(\d{4})",        # ter1996
        r"^(\d{4})_all",      # 2000_all
        r"_(\d{4})$",         # *_2007
        r"_(\d{4})_mcp",      # *_2007_mcp
    ]
    for pat in filename_patterns:
        m = re.search(pat, filename)
        if m:
            y = int(m.group(1))
            if 1995 <= y <= 2022:
                return y

    path_patterns = [
        r"/(\d{4})\s*mcps?/",       # /2013 mcps/
        r"/(\d{4})\s*MCPs?/",       # /2014 MCPs/
        r"/(\d{4})\s*Wolf[^/]*/",   # /2021 Wolf Territory.../
        r"/(\d{4})_Wolf[^/]*/",     # /2022_Wolf Territory.../
        r"/(\d{4})/",               # bare /2008/ directory
    ]
    for pat in path_patterns:
        m = re.search(pat, s, re.IGNORECASE)
        if m:
            y = int(m.group(1))
            if 1995 <= y <= 2022:
                return y

    all_years = re.findall(r"(?<![/_])(\d{4})(?![/_\d])", s)
    for ys in reversed(all_years):
        y = int(ys)
        if 1995 <= y <= 2022:
            return y

    return None

def find_year_shapefiles(root: Path, year: int):
    out = []
    for shp in root.rglob("*.shp"):
        y = extract_year_from_path(shp)
        if y == year:
            out.append(shp)
    return out

for year in range(START_YEAR, END_YEAR + 1):
    comp_path = COMPOSITE_DIR / f"wolfcast_{year}.tif"
    if not comp_path.exists():
        print(f"[skip] no composite for {year}: {comp_path}")
        continue

    shp_paths = find_year_shapefiles(SHAPE_ROOT, year)
    if not shp_paths:
        print(f"[skip] no shapefiles for {year}")
        continue

    geoms = []
    for shp in shp_paths:
        gdf = gpd.read_file(shp)
        if gdf.empty:
            continue
        if gdf.crs is None:
            gdf.set_crs("EPSG:26912", inplace=True)
        elif gdf.crs.to_epsg() != 26912:
            gdf = gdf.to_crs("EPSG:26912")
        geoms.extend(list(gdf.geometry))

    if not geoms:
        print(f"[skip] no valid geometries for {year}")
        continue

    with rasterio.open(comp_path) as src:
        profile = src.profile
        transform = src.transform
        out_shape = (src.height, src.width)

    mask = rasterize(
        [(g, 1) for g in geoms if g is not None],
        out_shape=out_shape,
        transform=transform,
        fill=0,
        dtype="uint8",
        all_touched=False,
    )

    out_path = OUTPUT_DIR / f"wolf_presence_{year}.tif"
    profile_out = profile.copy()
    profile_out.update({"count": 1, "dtype": "uint8", "nodata": 0})

    with rasterio.open(out_path, "w", **profile_out) as dst:
        dst.write(mask, 1)

    print(f"[ok] {year} -> {out_path}")

[ok] 1995 -> /content/drive/MyDrive/WolfCast_Satellite_Data_labels/wolf_presence_1995.tif
[ok] 1996 -> /content/drive/MyDrive/WolfCast_Satellite_Data_labels/wolf_presence_1996.tif
[ok] 1997 -> /content/drive/MyDrive/WolfCast_Satellite_Data_labels/wolf_presence_1997.tif
[ok] 1998 -> /content/drive/MyDrive/WolfCast_Satellite_Data_labels/wolf_presence_1998.tif
[ok] 1999 -> /content/drive/MyDrive/WolfCast_Satellite_Data_labels/wolf_presence_1999.tif
[ok] 2000 -> /content/drive/MyDrive/WolfCast_Satellite_Data_labels/wolf_presence_2000.tif
[ok] 2001 -> /content/drive/MyDrive/WolfCast_Satellite_Data_labels/wolf_presence_2001.tif
[ok] 2002 -> /content/drive/MyDrive/WolfCast_Satellite_Data_labels/wolf_presence_2002.tif
[ok] 2003 -> /content/drive/MyDrive/WolfCast_Satellite_Data_labels/wolf_presence_2003.tif
[ok] 2004 -> /content/drive/MyDrive/WolfCast_Satellite_Data_labels/wolf_presence_2004.tif
[ok] 2005 -> /content/drive/MyDrive/WolfCast_Satellite_Data_labels/wolf_presence_2005.tif
[ok] 2006 

# Make training data
Sample pixels from the tifs for each year at a given ration of positive to negative samples.

In [ ]:
"""
Build training table from Drive TIFFs.

Samples pixels from yearly composites + presence masks + NLCD,
writes a Parquet table with features and labels for Spark training.
"""

from pathlib import Path

import numpy as np
import pandas as pd
import rasterio

DRIVE_ROOT = Path("/content/drive/MyDrive")

COMPOSITE_DIR = DRIVE_ROOT / "WolfCast_Satellite_Data"
LABEL_DIR = DRIVE_ROOT / "WolfCast_Satellite_Data_labels"
NLCD_PATH = DRIVE_ROOT / "WolfCast_Satellite_Data" / "wolfcast_nlcd_2019.tif"

OUTPUT_DIR = DRIVE_ROOT / "81m_1neg_1pos"

START_YEAR = 1995
END_YEAR = 2022

NEGATIVE_RATIO = 1.0
MAX_POSITIVES_PER_YEAR = 1500000

BAND_NAMES = ["BLUE", "GREEN", "RED", "NIR", "SWIR1", "SWIR2"]
INDEX_NAMES = ["NDVI", "EVI", "NDWI", "NBR", "BSI", "NDSI"]


def sample_pixels_for_year(year: int):
    comp_path = COMPOSITE_DIR / f"wolfcast_{year}.tif"
    label_path = LABEL_DIR / f"wolf_presence_{year}.tif"

    if not comp_path.exists() or not label_path.exists():
        print(f"[skip] missing files for {year}")
        return None

    with rasterio.open(comp_path) as comp_src, rasterio.open(label_path) as label_src, rasterio.open(NLCD_PATH) as nlcd_src:
        comp_data = comp_src.read()
        label_data = label_src.read(1)
        nlcd_data = nlcd_src.read(1)

        h, w = label_data.shape
        comp_data = comp_data[:, :h, :w]
        nlcd_data = nlcd_data[:h, :w]

        n_pixels = h * w
        comp_flat = comp_data.reshape(len(comp_data), n_pixels).T
        label_flat = label_data.flatten()
        nlcd_flat = nlcd_data.flatten()

        valid = ~np.isnan(comp_flat).any(axis=1) & (nlcd_flat > 0)

        if not valid.any():
            print(f"[skip] no valid pixels for {year}")
            return None

        comp_valid = comp_flat[valid]
        label_valid = label_flat[valid]
        nlcd_valid = nlcd_flat[valid]

        pos_mask = label_valid == 1
        pos_indices = np.where(pos_mask)[0]
        if len(pos_indices) > MAX_POSITIVES_PER_YEAR:
            pos_indices = np.random.choice(pos_indices, MAX_POSITIVES_PER_YEAR, replace=False)
        n_pos = len(pos_indices)

        neg_mask = label_valid == 0
        neg_indices = np.where(neg_mask)[0]
        n_neg_target = int(n_pos * NEGATIVE_RATIO)
        if len(neg_indices) > n_neg_target:
            neg_indices = np.random.choice(neg_indices, n_neg_target, replace=False)

        all_indices = np.concatenate([pos_indices, neg_indices])
        np.random.shuffle(all_indices)

        rows = []
        for idx in all_indices:
            row = {
                "year": year,
                "presence": int(label_valid[idx]),
                "nlcd_class": int(nlcd_valid[idx]),
            }
            for i, name in enumerate(BAND_NAMES):
                row[name] = float(comp_valid[idx, i])
            for i, name in enumerate(INDEX_NAMES):
                row[name] = float(comp_valid[idx, i + len(BAND_NAMES)])
            rows.append(row)

        df = pd.DataFrame(rows)
        print(f"[ok] {year}: {len(df)} samples ({df['presence'].sum()} pos, {len(df) - df['presence'].sum()} neg)")
        return df


def main():
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

    total_pos = 0
    total_neg = 0

    for year in range(START_YEAR, END_YEAR + 1):
        df = sample_pixels_for_year(year)
        if df is None:
            continue

        n_pos = df['presence'].sum()
        n_neg = len(df) - n_pos
        total_pos += n_pos
        total_neg += n_neg

        # Write each year to separate file
        year_path = OUTPUT_DIR / f"samples_{year}.parquet"
        df.to_parquet(year_path, index=False)
        del df  # free memory immediately

    print(f"\n[total] {total_pos + total_neg} samples")
    print(f"  Positives: {total_pos}")
    print(f"  Negatives: {total_neg}")
    print(f"\n[saved] {OUTPUT_DIR}/*.parquet")
    print("Spark can read the whole directory with: spark.read.parquet('path/to/wolf_training_samples')")


if __name__ == "__main__":
    main()



[ok] 1995: 3000000 samples (1500000 pos, 1500000 neg)
[ok] 1996: 3000000 samples (1500000 pos, 1500000 neg)
[ok] 1997: 3000000 samples (1500000 pos, 1500000 neg)
[ok] 1998: 3000000 samples (1500000 pos, 1500000 neg)
[ok] 1999: 3000000 samples (1500000 pos, 1500000 neg)
[ok] 2000: 3000000 samples (1500000 pos, 1500000 neg)
[ok] 2001: 3000000 samples (1500000 pos, 1500000 neg)
[ok] 2002: 3000000 samples (1500000 pos, 1500000 neg)
